In [3]:
import os
import re
import random

import numpy as np

# import image_encoder
# from image_encoder import encode_standard_array

In [4]:
import cv2
import tensorflow as tf
import numpy as np

# from MBnet_test import mobilenet_v3_small
from MBnet_val import mobilenet_v3_small

def encode_standard_array(img_path):

    im = cv2.imread(img_path)
    # print(im.shape)
    im = cv2.resize(im, (224, 224), interpolation=cv2.INTER_CUBIC)
    # print(im.shape)
    img_array = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    # print(x_train.shape)
    img_array=img_array[:,:,np.newaxis]
    img_array=img_array[np.newaxis,:,:,:]/255.0

    return img_array

def embed_shoe(img_array, model_path="./mobile_checkpoint/new_mobile_goal.ckpt"):

#     img_array = encode_standard_array(path)
    tf.reset_default_graph()
    
    with tf.name_scope('input'):
            inputs = tf.placeholder(tf.float32, [None, 224, 224, 1]) ##輸入為四維[Batch_size,height,width,channels] 
    with tf.name_scope('stem'):

        # out_triplet,out_softmax = MobileV3(inputs)
        softmax, triplet, end_points = mobilenet_v3_small(
            inputs, # input_test
            103,
            multiplier=1.0, 
            is_training=True, 
            reuse=None
        ) # model: softmax outcome
        
    saver = tf.train.Saver()

    with tf.Session() as sess:
            
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, model_path) 
        pred = sess.run(
                        [triplet], 
                        feed_dict={inputs:img_array}) #list

        pred = pred[0]    

    return pred

In [5]:

# base_dir = '/home/jsamare01/DML/dataset/downloads/' # v1 test

base_dir = '/home/troy0425/data/collect_new/images_all/'
shoe_dirs_total = ["{}{}/".format(base_dir, dir_) for dir_ in os.listdir(base_dir) 
          if os.path.isdir(os.path.join(base_dir, dir_))]

random.seed(4428)
print("Shoes class",len(shoe_dirs_total))
random_idx = random.sample(range(len(shoe_dirs_total)), 103)

# first_shoe_dirs = [shoe_dirs_total[idx] for idx in range(len(shoe_dirs_total)) if 
#               idx in random_idx]
# second_shoe_dirs = [shoe_dirs_total[idx] for idx in range(len(shoe_dirs_total)) if 
#               idx not in random_idx]

# random.seed(4428)
# shoe_dirs_total = random.sample(shoe_dirs_total, 150)

sample_dir = [shoe_dirs_total[idx] for idx in random_idx]
shoe_dirs_total = sample_dir

all_img_path = []
for dir_ in shoe_dirs_total:
    img_path = ["{}{}".format(dir_, path) for path in os.listdir(dir_)
                if path.endswith(".jpg")]

    for path in img_path:
        info = path.split('/')
        label = "{}-{}".format(info[-2], re.sub(".jpg", "", info[-1]))
        all_img_path.append(
            {"line": info[-2], "label": label, "path": path, "array": None}
        ) # label, path, array

print("Total {} lines with {} shoes.".format(len(shoe_dirs_total), len(all_img_path)))
labels = [dir_.split('/')[-2] for dir_ in shoe_dirs_total]

# encode all data into array
%time all_array = list(map(lambda img_info: encode_standard_array(img_info.get('path')),all_img_path))
img_matrix = np.concatenate(all_array)
print("Embedding....")
%time emb_matrix = embed_shoe(img_matrix)


# save embedding vector
embedding_vector_path = "embedding_vector_496.npy"
np.save(embedding_vector_path, emb_matrix)
# embedding_vector_path = "embedding_vector_496.npy"
# img_matrix = np.load(embedding_vector_path)

Shoes class 496
Total 103 lines with 10405 shoes.
CPU times: user 1min 3s, sys: 2.22 s, total: 1min 5s
Wall time: 1min 5s


W0730 06:40:10.301486 140221308585728 deprecation_wrapper.py:119] From /home/yu-theturtle/MBnet_val.py:207: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0730 06:40:10.303858 140221308585728 deprecation.py:506] From /home/yu-theturtle/tutorial4/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Embedding....
inputs Tensor("input/Placeholder:0", shape=(?, 224, 224, 1), dtype=float32)
input_size [224, 224]


W0730 06:40:11.117968 140221308585728 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0730 06:40:11.119240 140221308585728 deprecation.py:323] From /home/yu-theturtle/MBnet_val.py:62: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0730 06:40:11.358977 140221308585728 deprecation.py:323] From /home/yu-theturtle/MBnet_val.py:53: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular

CPU times: user 8min 18s, sys: 1min 27s, total: 9min 45s
Wall time: 3min 15s


In [6]:
import gc
first_emb_matrix = embed_dir_images(first_shoe_dirs)
gc.collect()
second_emb_matrix = embed_dir_images(second_shoe_dirs)

NameError: name 'first_shoe_dirs' is not defined

In [7]:
np.concatenate((first_emb_matrix, second_emb_matrix)).shape

NameError: name 'first_emb_matrix' is not defined

In [ ]:
%time emb_matrix = embed_shoe(img_matrix)

sample_path = "all_embeddind_100line_sample_v2.npy"

np.save(sample_path, emb_matrix)
emb_matrix = np.load(sample_path)

print(emb_matrix.shape)
print(len(all_img_path))

In [ ]:
embed_shoe(all_array[0])

In [ ]:
for array, img_info in zip(emb_matrix, all_img_path):
    img_info['array'] = array

In [ ]:
print(len(all_img_path))
all_img_path[0]

In [ ]:
''' each line with its embedding matrix from image '''
line_df = []
all_line = [line_str.split('/')[-2] for line_str in shoe_dirs_total]
for line in all_line:
    target_info_list = [img_info.get('array') for img_info in all_img_path 
                     if img_info.get('path').split('/')[-2] == line]
    all_matrix = np.vstack(target_info_list)
    line_df.append({"label": line, "emb_matrix": all_matrix})

In [ ]:
len(line_df)

## Keyword Image Query

In [16]:
import requests
from bs4 import BeautifulSoup

ImportError: No module named bs4

In [17]:
def crawl_img_url(keyword):
    '''
    Input : keyword in string format
    Output: url in string format
    '''
# keyword = "Nike Air Max"
    keyword = "+".join([token.lower() for token in keyword.split(' ')])
    query = "https://www.google.com/search?biw=1280&bih=688&tbs=isz%3Ai&tbm=isch&sa=1&ei=pKs9XdC0H6WImAWYmrXgCg&q={}x&oq={}&gs_l=img.12...0.0..7026...0.0..0.0.0.......0......gws-wiz-img.0DrM6W-zvnU&ved=0ahUKEwjQlryi5NfjAhUlBKYKHRhNDawQ4dUDCAY".format(
            keyword, keyword)

    page = requests.get(query).text
    soup = BeautifulSoup(page, 'html.parser')
    img_url = soup.find("img")['src']
    
    return img_url

In [18]:
shoe_url = crawl_img_url('Nike SB Dunk High Elite-8641')
print(shoe_url)

NameError: global name 'BeautifulSoup' is not defined

## Chinese Whispers

In [19]:
def face_distance(face_encodings, face_to_compare):
    """
    计算一组特征值与带比较特征值之间的距离，默认采用欧氏距离
    参数配置
    face_encodings:一组特征值，包含多个
    face_to_compare:待比较特征值，只有一个
    return:返回不同特征向量之间距离的数组矩阵
    """
    import numpy as np
    if len(face_encodings) == 0:
        return np.empty((0))
    '''
    利用numpy包进行距离估量
    http://blog.csdn.net/u013749540/article/details/51813922
    '''
    dist=[]

    """
    # 欧氏距离，考虑后续邻接边选择weight较大者，此处选取余弦相似度
    for i in range(0,len(face_encodings)):
        #sim = 1.0/(1.0+np.linalg.norm(face_encodings[i]-face_to_compare))
        sim=np.linalg.norm(face_encodings[i]-face_to_compare)
        dist.append(sim)
    """
    # 余弦相似度
    for i in range(0, len(face_encodings)):
        num=np.dot(face_encodings[i],face_to_compare)
        cos=num/(np.linalg.norm(face_encodings[i])*np.linalg.norm(face_to_compare))
        sim=0.5+0.5*cos # 归一化
        dist.append(sim)
    return dist

def find_all_index(arr,item):
    '''获取list中相同元素的索引
    输入：
        arr：待求取list
        item：待获取元素

    输出：
        相同元素索引，格式为list'''
    return [i for i, a in enumerate(arr) if a==item]

def _chinese_whispers(emb_matrix, threshold=0.675, iterations=10):
    """ Chinese Whisper Algorithm
    算法概要
        1.初始化每个节点为一个类
        2.选取任意节点开始迭代
            选择该节点邻居中边权重最大者，将两则归为一类；若邻居中有2者以上属于同一类，将这些类权重相加进行比较

    输入：
        encoding_list:待分类的特征向量组
        threshold:判断门限，判断两个向量是否相关
        iteration:迭代次数

    输出：
        sorted_clusters:一组分类结果，按从大到小排列
    """

    from random import shuffle
    import networkx as nx
    import numpy as np
    import re
    print('Clustering With {} embeddings.'.format(emb_matrix.shape[0]))
    # Create graph
    nodes = []
    edges = []

    #encoding_list格式为
    #[(path1,encode1),(path2,encode2),(path3,encode3)]
    #image_paths, encodings = zip(*encoding_list)
    feature_matrix=emb_matrix
    encodings=[]
    #image_paths=[]
    for i in range(0,len(feature_matrix)):
        encodings.append(feature_matrix[i,:])
        #image_paths.append(r'F:\outCluster\%d\\' %i)

    if len(encodings) <= 1:
        print ("No enough encodings to cluster!")
        return []

    ''' 
    节点初始化：
        1.将每个特征向量设为一个类
        2.计算每个特征向量之间的距离，并根据门限判定是否构成邻接边
    '''
    local_farest = []
    local_nearest = []
    
    for idx, face_encoding_to_check in enumerate(encodings):
        # Adding node of facial encoding
        node_id = idx

        # 节点属性包括
        # node_id:节点id,(0,n-1)
        # label:节点类别，初始化每个节点一个类别
        # path：节点导出路径，用于图片分类导出
        node = (node_id, {'label':idx})
        #node = (node_id, {'label': idx, 'path': image_paths[idx]})
        nodes.append(node)

        # Facial encodings to compare
        if (idx+1) >= len(encodings):
            # Node is last element, don't create edge
            break

        #构造比较向量组
        #若当前向量为i,则比较向量组为[i+1:n]
        compare_encodings = encodings[idx+1:]
        distances = face_distance(compare_encodings, face_encoding_to_check)
        encoding_edges = []
        for i, distance in enumerate(distances):
            # 若人脸特征匹配，则在这两个节点间添加关联边
            if distance >= threshold:
                #edge_id：与node_id相连接的节点的node_id
                edge_id = idx+i+1
                encoding_edges.append((node_id, edge_id, {'weight': distance}))
        local_farest.append(np.max(distance))
        local_nearest.append(np.min(distance))
        edges = edges + encoding_edges

    print("Pointwise Distance Max Farest: {}".format(max(local_farest)))
#     print('Mean Farest: {}'.format(np.mean(local_farest)))
#     print("Min Farest: {}".format(min(local_farest)))

#     print("Max Nearest: {}".format(max(local_nearest)))
#     print('Mean Nearest: {}'.format(np.mean(local_nearest)))
    print("Pointwise Distance Min Nearest: {}".format(min(local_nearest)))
    print("Farest - Nearest Distance: {}".format(max(local_farest) - min(local_nearest)))
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)

    '''
    迭代过程

    '''
    for _ in range(0, iterations):
        cluster_nodes = list(G.nodes()) #返回节点id
        shuffle(cluster_nodes)# 随机选取一个开始节点
        for node in cluster_nodes:
            # 当前节点的所有邻接边，如节点4邻接边为(4,5,weight=8)(4,8,weight=10)
            # 则G[4]返回值为AtlasView({5:{'weight':8}, 8:{'weight':10}})
            neighbors = G[node]
            # cluster形式
            # {'cluster_path':weight}   其中cluster_paht=node属性的cluster值
            labels = {}

            for ne in neighbors: # ne即为当前节点邻接的节点id
                if isinstance(ne, int):
                    '''
                    判断该邻居的类别是否在其他邻居中存在
                        若存在，则将相同类别的权重相加。
                    '''
                    if G.node[ne]['label'] in labels:#G.node[ne]['label']即为id=ne节点的label属性
                        labels[G.node[ne]['label']] += G[node][ne]['weight']#将这条邻接边(node,ne)的weight属性赋值给cluster[节点ne的cluster]
                    else:
                        labels[G.node[ne]['label']] = G[node][ne]['weight']

            # find the class with the highest edge weight sum
            edge_weight_sum = 0
            max_cluster = 0
            #将邻居节点的权重最大值对应的文件路径给到当前节点
            #这里cluster即为path
            for id in labels:
                if labels[id] > edge_weight_sum:
                    edge_weight_sum = labels[id]
                    max_cluster = id

            # set the class of target node to the winning local class
            #print('node %s was clustered in %s' %(node, max_cluster))
            G.node[node]['label'] = max_cluster
    list_label_out = []
    for i in range(len(encodings)):
        list_label_out.append(G.node[i]['label'])
    #print(list_label_out)

    ''' 

    统计分类错误数量=新类别中不属于原类别的数量      eg： list_label_out=[1,3,4,2,2,4,3,1]
    # group_all 返回最终类别标签                     group_all=[1,2,3,4]
    # group_num 最终分类数量                        group_num=4
    # group_cluster: list,返回相同标签的节点id       group_cluster=[[0,7],[3,4],[1,6],[2,5]]
    '''
    group_all = set(list_label_out)
    group_num = len(group_all)
    group_cluster = []

    for item in group_all:
        group_cluster.append(find_all_index(list_label_out,item))

    centroid_list = []
    print('最終分類數量：%s' %group_num)
    for i in range(0,group_num):
        print('第%d類：%s'%(i,group_cluster[i]))
        centroid = emb_matrix[group_cluster[i]].mean(axis=0)
        centroid_list.append(centroid)
    print('')
    
    
    return np.vstack(centroid_list)

In [20]:
# import cv2

# def variance_of_laplacian(image):
#     # compute the Laplacian of the image and then return the focus
#     # measure, which is simply the variance of the Laplacian
#     return cv2.Laplacian(image, cv2.CV_64F).var()

In [21]:
# emb_var_matrix = []
# for idx in range(100):
#     test_emb_var = line_df[idx].get('emb_matrix').var(axis=0)
#     print("Mean: {}".format(np.mean(test_emb_var)))
#     print("Var: {} \n".format(test_emb_var.var()))
#     emb_var_matrix.append(test_emb_var)
    
# emb_var_df = pd.DataFrame(emb_var_matrix)
# emb_var_df.shape

In [22]:
emb_var_df

NameError: name 'emb_var_df' is not defined

In [23]:
for idx, line_info in enumerate(line_df):
    print('For {} line {}'.format(idx, line_info.get('label')))
    local_emb_var = line_info.get('emb_matrix').var(axis=0)
#     print("Mean: {}".format(np.mean(test_emb_var)))
    local_threshold = np.mean(local_emb_var)
    print("Threshold: {}".format(local_threshold))
    centroid_list = _chinese_whispers(line_info.get('emb_matrix'), 
                     threshold= 0.6,
                     iterations= 100)
#     print(centroid_list.shape)
    line_info["centroids"] = centroid_list

For 0 line Nike Zoom Kobe 7


TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [18]:
''' Save File of each embedding '''
import pandas as pd 
cent_df = []
for info in line_df:
    for cent in info.get('centroids'):
         cent_df.append({"line": info.get('label'), "centroid": cent})
cent_df = pd.DataFrame(cent_df)

NameError: name 'line_df' is not defined

In [ ]:
cent_df.head(5)

In [ ]:
centro_matrix = np.array(cent_df.centroid.tolist())

In [ ]:
centro_matrix.shape

In [ ]:
# np.save("50_lines_centroids.npy", centro_matrix)
# cent_df.line.to_csv('50_line_name')

## Testing Find the most similar

In [ ]:
cent_df.loc[cent_df.line.apply(lambda line: "Nike Air Huarache" in line)]

In [ ]:
test_img_path = 'Nike Huarache Trainer Free.jpg'
img_array = encode_standard_array(test_img_path)
# img_array = (img_array/255.0)
test_array = embed_shoe(img_array)

In [25]:
test_array

NameError: name 'test_array' is not defined

In [58]:
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix = cosine_similarity(test_array, centro_matrix)[0]

NameError: name 'centro_matrix' is not defined

In [ ]:
sim_matrix[sim_matrix.argsort()[-1]]

In [ ]:
sim_rank = list(sim_matrix.argsort()[-11:][::-1])

In [ ]:
list(sim_rank)

In [ ]:
for idx, rank in enumerate(sim_rank):
    print('The {} candidates is {}'.format(idx, cent_df.iloc[rank].line))

## Random Select photo to compare with top3

In [1]:
# shoe_dirs_total = sample_dir

testing_img_path = []
rdm_num = 10
for dir_ in shoe_dirs_total:
    img_path = ["{}{}".format(dir_, path) for path in os.listdir(dir_)
                if path.endswith(".jpg")]
    img_path = random.sample(img_path, rdm_num)
    for path in img_path:
        info = path.split('/')
        label = "{}-{}".format(info[-2], re.sub(".jpg", "", info[-1]))
        testing_img_path.append(
            {"line": info[-2], "label": label, "path": path, "array": None}
        ) # label, path, array

print("Total {} lines with {} shoes.".format(len(shoe_dirs_total), len(testing_img_path)))
# testing_labels = [dir_.split('/')[-2] for dir_ in shoe_dirs_total]

# encode all data into array

%time testing_array = list(map(lambda img_info: encode_standard_array(img_info.get('path')),testing_img_path))
testing_img_matrix = np.concatenate(testing_array)
%time testing_emb_matrix = embed_shoe(testing_img_matrix)

NameError: name 'shoe_dirs_total' is not defined

In [44]:
shoe_dirs_total

['/home/troy0425/data/collect_new/images/Nike Air Footscape Woven/',
 '/home/troy0425/data/collect_new/images/Nike Air Penny 2/',
 '/home/troy0425/data/collect_new/images/Nike Air Up/',
 '/home/troy0425/data/collect_new/images/Nike Air Bo 1/',
 '/home/troy0425/data/collect_new/images/Nike LunarGlide+ 3/',
 '/home/troy0425/data/collect_new/images/Nike SB Eric Koston 2/',
 '/home/troy0425/data/collect_new/images/Nike Air Tech Challenge Hybrid/',
 '/home/troy0425/data/collect_new/images/Nike Air Max CB34 II/',
 '/home/troy0425/data/collect_new/images/Nike Flight 13/',
 '/home/troy0425/data/collect_new/images/Nike Free Flyknit/',
 '/home/troy0425/data/collect_new/images/Nike Hyperdunk 2012/',
 '/home/troy0425/data/collect_new/images/Nike React Element 55/',
 '/home/troy0425/data/collect_new/images/Nike Air Max Light/',
 '/home/troy0425/data/collect_new/images/Nike Free 3.0 V3/',
 '/home/troy0425/data/collect_new/images/Nike Air Max Hyperaggressor/',
 '/home/troy0425/data/collect_new/images

In [45]:
test_img_path = 'Nike KD 8 test.jpg'
img_array = encode_standard_array(test_img_path)
# img_array = (img_array/255.0)
test_array = embed_shoe(img_array)

inputs Tensor("input/Placeholder:0", shape=(?, 224, 224, 1), dtype=float32)
input_size [224, 224]


In [46]:
testing_emb_matrix

array([[  4.524671  ,   3.114753  ,   2.2914023 , ...,  -6.759872  ,
         -3.70871   ,   0.6850882 ],
       [  1.7791815 ,  -0.8430152 , -22.551268  , ..., -19.224007  ,
         -0.5281583 ,  -4.8942475 ],
       [ -1.4716938 ,  21.651424  ,  -0.90004283, ...,  18.32589   ,
        -39.80056   , -27.024124  ],
       ...,
       [-11.117239  ,  18.874275  , -15.765719  , ...,  19.803226  ,
         -5.683696  ,  11.248133  ],
       [ -7.256352  ,  -8.146556  ,  -2.2535844 , ...,  18.97243   ,
         -8.077344  , -23.640623  ],
       [ -8.662808  ,  -7.941438  , -17.01536   , ...,   9.497304  ,
         -1.985655  ,  -1.9609196 ]], dtype=float32)

In [64]:
# np.apply_along_axis(lambda x: , 0, test_array)
similarity_list = []
local_min_list = []

local_simi_list = []
for idx, emb in enumerate(testing_emb_matrix):
    if (idx % rdm_num == 0) and (idx != 0): 
#         local_simi_list = []
        
        #local_min_list.append(np.mean(local_simi_list))
        local_simi_list.sort()
        local_simi_list = local_min_list[:10]
        local_min_list.append(np.mean(local_simi_list))
        local_simi_list = []
        
    similarity = np.linalg.norm(emb - test_array)
    
#     print("Distance to {}: {}".format(line_list[idx], similarity))
#     similarity_list.append(similarity)
    local_simi_list.append(similarity)
    
local_min_list.append(np.mean(local_simi_list))    
# similarity_list = np.array(similarity_list)
# simi_rank = similarity_list.argsort()[-30:][::-1]

print(local_min_list)

[161.03671, 135.51222, 206.38152, 146.15671, 158.01987, 116.69546, 154.05438, 236.15955, 97.88451, 154.12987, 107.325455, 112.4235, 163.80966, 153.44035, 154.76183, 148.61826, 122.91362, 144.8728, 129.66614, 149.48132, 133.86456, 238.87549, 117.51796, 223.01811, 177.34512, 151.32593, 168.51779, 175.75314, 140.45786, 127.32655, 134.70435, 144.93622, 158.5477, 132.51534, 253.60214, 149.15485, 129.0585, 164.32376, 240.5312, 119.64177, 145.50797, 172.42046, 156.15817, 108.14319, 186.62302, 137.62701, 183.42642, 177.9924, 120.16632, 187.19756, 126.30847, 97.58478, 172.0042, 191.41177, 121.1083, 187.58113, 177.1094, 223.43344, 123.45354, 128.83255, 178.4444, 121.87982, 171.66072, 163.16068, 147.39055, 133.84857, 95.742874, 175.87498, 125.11238, 125.84275, 207.41458, 219.19919, 198.04279, 119.49104, 161.6844, 134.83304, 176.46767, 115.50924, 128.17838, 158.61844, 107.17969, 135.28328, 177.48628, 156.13171, 173.70137, 110.77806, 126.37565, 113.44563, 117.42228, 119.510056, 115.444725, 110.4301

In [63]:
dist_df = []
for idx, distance in enumerate(local_min_list):
#     print("Distance for {}: {}".format((shoe_dirs_total[i`x]).split('/')[-2], 
#                                        distance))
    dist_df.append({"Name": (shoe_dirs_total[idx]).split('/')[-2], 
                    "Similarity": distance})
    
dist_df = pd.DataFrame(dist_df)
dist_df.sort_values(by=['Similarity']).reset_index()

,index,Name,Similarity
0,59,Nike Air Footscape Magista,0.389345
1,72,Nike ACG 07 KMTR,0.391484
2,35,Nike Hyperdunk 2010,0.397950
3,88,Nike ACG Ruckel Ridge,0.400407
4,26,Nike Hyperdunk 2016,0.410608
5,11,Nike React Element 55,0.415401
6,87,Nike Zoom Field General,0.417247
7,96,Nike Air Unlimited,0.421231
8,98,Nike LunarEpic Flyknit,0.425396
9,97,Nike Shox TLX,0.428262


In [37]:
for idx, rank in enumerate(simi_rank):
    print('{} Candidates: {}'.format(idx, line_list[rank]))

NameError: name 'simi_rank' is not defined

In [38]:
testing_emb_matrix(np.linalg.norm(a-b))

NameError: name 'testing_emb_matrix' is not defined

In [39]:
testing_simi_matrix = cosine_similarity(test_array, testing_emb_matrix)[0]
testing_rank = (testing_simi_matrix.argsort())[-11:][::-1]

NameError: name 'test_array' is not defined

In [40]:
len(testing_simi_matrix)

NameError: name 'testing_simi_matrix' is not defined

In [41]:
line_list = list(np.repeat(shoe_dirs_total, 10))
line_list = [line.split('/')[-2] for line in line_list]

In [42]:
for idx, rank in enumerate(testing_rank):
    print('The {} candidates is {}'.format(idx, line_list[rank]))

NameError: name 'testing_rank' is not defined

In [46]:
emb_list = []
for img_idx in ['1', '2', '3', '4']:
    file_path = "Nike Air Penny {}.jpg".format(img_idx)
    emb = embed_shoe(encode_standard_array(file_path))
    emb_list.append(emb)

NameError: name 'embed_shoe' is not defined

In [44]:
emb_list = np.array(emb_list).reshape(4, 128)

ValueError: cannot reshape array of size 0 into shape (4,128)

In [45]:
cosine_similarity(np.array(emb_list))

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.